<a href="https://colab.research.google.com/github/alexiisvs/E2-ARCHIVOS/blob/main/t2_gop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pregunta A

2) Para este apartado primero se debe de instalar Gurobi

In [1]:
!pip install gurobipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 27.8 MB/s eta 0:00:00


In [2]:
import gurobipy as gp
from gurobipy import Model, GRB
import scipy as sp
import numpy as np
import pandas as pd


In [3]:
# 1. Datos
T = [1, 2, 3, 4, 5]

kb = {1: 30,  2: 60,  3: 50,  4: 40,  5: 30}   # kg ropa buen estado entrantes
km = {1: 25,  2: 50,  3: 45,  4: 20,  5: 35}   # kg ropa mal estado entrantes
D  = {1: 280, 2: 320, 3: 310, 4: 270, 5: 290}  # demanda (unidades de prenda)


# 2. Parámetros adicionales (sección Parámetro–Valor)
r_b   = 10     # inventario inicial buen estado (kg)
r_m   = 20     # inventario inicial mal estado (kg)
w0    = 1      # trabajadores fijos iniciales
h     = 8      # horas por trabajador en el período
c_c   = 12000  # costo hora normal total de w0 trabajadores ($/período)
c_t   = 210000 # costo contratación por trabajador (por boleta) ($/persona·período)
c_p   = 6000   # penalización por prenda no entregada ($/prenda)
g     = 380    # costo transformar mal→género ($/kg)
n     = 260    # costo confeccionar género→prenda ($/kg)
a     = 420    # costo almacenaje ($/kg·período)
s     = 450    # capacidad máxima de almacén (kg)
p     = 0.5    # kg de tela requerido por prenda
tau_g = 0.18   # horas/kg para mal→género
tau_n = 0.22   # horas/kg para género→prenda


# 3. Construcción del modelo
m = Model("Transformacion_Textil")

# Variables de decisión
I_b = m.addVars(T, lb=0.0, name="I_b")   # inv. final buen estado (kg)
I_m = m.addVars(T, lb=0.0, name="I_m")   # inv. final mal estado (kg)
I_g = m.addVars(T, lb=0.0, name="I_g")   # inv. final género textil (kg)

x_b = m.addVars(T, lb=0.0, name="x_b")   # kg usados de buen estado
x_g = m.addVars(T, lb=0.0, name="x_g")   # kg de género convertidos a prenda
kgg = m.addVars(T, lb=0.0, name="kgg")   # kg procesados de mal → género
u   = m.addVars(T, lb=0.0, name="u")     # prendas no satisfechas
y   = m.addVars(T, vtype=GRB.INTEGER, lb=0, name="y")  # nuevos contratados

# Objetivo: minimizar costos
m.setObjective(
    sum(
        # costo fijo de los w0 trabajadores
        c_c
        # costo contrataciones
        + c_t * y[t]
        # transformación + confección
        + g * kgg[t]
        + n * x_g[t]
        # almacenamiento
        + a * (I_b[t] + I_m[t] + I_g[t])
        # penalización incumplimiento
        + c_p * u[t]
        for t in T
    ),
    GRB.MINIMIZE
)


# 4. Restricciones
for idx, t in enumerate(T):
    # 4.1 Balance ropa buen estado
    m.addConstr(r_b + kb[t] - x_b[t] == I_b[t], name=f"Bal_Buen_{t}")

    # 4.2 Balance ropa mal estado
    prev_Im = r_m if idx == 0 else I_m[T[idx-1]]
    m.addConstr(prev_Im + km[t] - kgg[t] == I_m[t], name=f"Bal_Mal_{t}")

    # 4.3 Balance género textil
    m.addConstr(kgg[t] - x_g[t] == I_g[t], name=f"Bal_Genero_{t}")

    # 4.4 Satisfacción de demanda
    m.addConstr(x_b[t]/p + x_g[t]/p + u[t] == D[t], name=f"Demanda_{t}")

    # 4.5 Horas-hombre disponibles
    m.addConstr(x_g[t]*tau_n + kgg[t]*tau_g <= (w0 + y[t]) * h, name=f"Horas_{t}")

    # 4.6 Capacidad de almacén
    m.addConstr(I_b[t] + I_m[t] + I_g[t] <= s, name=f"Almacen_{t}")


# 5. Optimizar e imprimir resultados
m.optimize()

if m.status == GRB.OPTIMAL:
    print("\n--- Resultados óptimos ---")
    for t in T:
        print(
            f"Período {t}: "
            f"I_b={I_b[t].X:.2f}, I_m={I_m[t].X:.2f}, I_g={I_g[t].X:.2f}, "
            f"x_b={x_b[t].X:.2f}, x_g={x_g[t].X:.2f}, kgg={kgg[t].X:.2f}, "
            f"u={u[t].X:.2f}, y={int(y[t].X)}"
        )
else:
    print("No se encontró solución óptima. Estado:", m.status)


Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 30 rows, 40 columns and 84 nonzeros
Model fingerprint: 0x6800c912
Variable types: 35 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-01, 8e+00]
  Objective range  [3e+02, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 4e+02]
Presolve removed 17 rows and 23 columns
Presolve time: 0.00s
Presolved: 13 rows, 17 columns, 36 nonzeros
Variable types: 12 continuous, 5 integer (0 binary)
Found heuristic solution: objective 8584800.0000

Root relaxation: objective 4.542300e+06, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth I

### 3) Repitiendo el modelo del inciso anterior:

In [ ]:


# Datos

T = [1, 2, 3, 4, 5]
kb = {1: 30,  2: 60,  3: 50,  4: 40,  5: 30}
km = {1: 25,  2: 50,  3: 45,  4: 20,  5: 35}
D  = {1: 280, 2: 320, 3: 310, 4: 270, 5: 290}


# Parámetros adicionales (inicialización)

r_b   = 10
r_m   = 20
w0    = 1
h     = 8
c_c   = 12000
c_t   = 210000
c_p   = 6000
g     = 380
n     = 260
a     = 420
s     = 450
p     = 0.5
tau_g = 0.18
tau_n = 0.22

def run_model(tau_g_param):
    m = Model()
    # Variables
    I_b = m.addVars(T, lb=0, name="I_b")
    I_m = m.addVars(T, lb=0, name="I_m")
    I_g = m.addVars(T, lb=0, name="I_g")
    x_b = m.addVars(T, lb=0, name="x_b")
    kgg = m.addVars(T, lb=0, name="kgg")
    x_g = m.addVars(T, lb=0, name="x_g")
    u   = m.addVars(T, lb=0, name="u")
    y   = m.addVars(T, vtype=GRB.INTEGER, name="y")
    # Objetivo
    m.setObjective(
        sum(
            c_c
            + c_t * y[t]
            + g * kgg[t]
            + n * x_g[t]
            + a * (I_b[t] + I_m[t] + I_g[t])
            + c_p * u[t]
            for t in T
        ), GRB.MINIMIZE
    )
    # Restricciones
    for idx, t in enumerate(T):
        prev_Im = r_m if idx == 0 else I_m[T[idx-1]]
        m.addConstr(r_b + kb[t]     - x_b[t]           == I_b[t])
        m.addConstr(prev_Im + km[t] - kgg[t]           == I_m[t])
        m.addConstr(kgg[t]          - x_g[t]           == I_g[t])
        m.addConstr(x_b[t] + x_g[t] + u[t]             == D[t])
        m.addConstr(x_g[t] * tau_n + kgg[t] * tau_g_param <= (w0 + y[t]) * h)
        m.addConstr(I_b[t] + I_m[t] + I_g[t]           <= s)
    m.optimize()

    # Recopilar resultados
    res = {
        'obj': m.ObjVal,
        'total_prod_prendas': sum((x_b[t].X + x_g[t].X) / p for t in T),
        'total_kgg': sum(kgg[t].X for t in T),
        'vars': {
            t: {'u': u[t].X, 'y': y[t].X}
            for t in T
        }
    }
    return res

# 1) Caso base
base = run_model(tau_g)

# 2) Inciso 3: aumentar tau_g en 25%
faulty = run_model(tau_g * 1.25)

# 3) Comparación de resultados
print("\n--- Comparación Base vs. Inciso 3 (τg ×1.25) ---")
print(f"Costo total: Base = ${base['obj']:.0f}  |  Inciso 3 = ${faulty['obj']:.0f}")
print(f"Producción total (prendas): Base = {base['total_prod_prendas']:.0f}  |  Inciso 3 = {faulty['total_prod_prendas']:.0f}")
print(f"Uso total de género (kg):        Base = {base['total_kgg']:.0f}  |  Inciso 3 = {faulty['total_kgg']:.0f}")
print("\nPeriodo | u_base | u_faulty | y_base | y_faulty")
for t in T:
    ub = base['vars'][t]['u'];   uf = faulty['vars'][t]['u']
    yb = base['vars'][t]['y'];   yf = faulty['vars'][t]['y']
    print(f"   {t}    | {ub:.0f}    | {uf:.0f}      | {yb:.0f}     | {yf:.0f}")


Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 30 rows, 40 columns and 84 nonzeros
Model fingerprint: 0x6d1bba3b
Variable types: 35 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-01, 8e+00]
  Objective range  [3e+02, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 4e+02]
Presolve removed 20 rows and 26 columns
Presolve time: 0.00s
Presolved: 10 rows, 14 columns, 30 nonzeros
Variable types: 9 continuous, 5 integer (0 binary)
Found heuristic solution: objective 1.215480e+07

Root relaxation: objective 6.924700e+06, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0         